In [11]:
import tensorflow as tf

tf.compat.v1.enable_v2_behavior()

In [7]:
import tensorflow_datasets as tfds

# # load datasets
alice_dataset, bob_dataset = tfds.load("fashion_mnist", split=["train[:50]","train[-50:]"])

Dl Completed...: 0 url [00:00, ? url/s]


Dl Completed...:  25%|██▌       | 1/4 [00:01<00:02,  1.03 url/s]

Dl Completed...:  50%|█████     | 2/4 [00:01<00:01,  1.22 url/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:26<00:08,  8.09s/ url]

Dl Completed...: 100%|██████████| 4/4 [02:11<00:00, 37.22s/ url]

Dl Completed...: 100%|██████████| 4/4 [02:12<00:00, 33.08s/ url]


Dataset fashion_mnist downloaded and prepared to C:\Users\ren\tensorflow_datasets\fashion_mnist\3.0.1. Subsequent calls will reuse this data.


In [12]:
import collections

BATCH_SIZE = 32


def cast(element):
    """Casts an image's pixels into float32."""
    out = {}
    out["image"] = tf.image.convert_image_dtype(element["image"], dtype=tf.float32)
    out["label"] = element["label"]
    return out


def flatten(element):
    """Flattens an image in preparation for the neural network."""
    return collections.OrderedDict(
        [
            ("x", tf.reshape(element["image"], [-1])),
            ("y", tf.reshape(element["label"], [1])),
        ]
    )


def preprocess(dataset):
    """Preprocesses images to be fed into neural network."""
    return dataset.map(cast).map(flatten).batch(BATCH_SIZE)

In [13]:
# batches of data
preprocessed_alice_dataset = preprocess(alice_dataset)
preprocessed_bob_dataset = preprocess(bob_dataset)

In [14]:
federated_data = [preprocessed_alice_dataset, preprocessed_bob_dataset]

In [15]:
from tensorflow.python.keras.optimizer_v2 import gradient_descent

LEARNING_RATE = 0.02


def custom_loss_function(y_true, y_pred):
    """Custom loss function."""
    #return tf.reduce_mean(
    #    tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    #)

    return tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    )


def create_compiled_keras_model():
    """Compiles the keras model."""
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Dense(
                10,
                activation=tf.nn.softmax,
                kernel_initializer="zeros",
                input_shape=(784,),
            )
        ]
    )
    model.compile(
        loss=custom_loss_function,
        optimizer=gradient_descent.SGD(learning_rate=LEARNING_RATE),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [17]:
batch_of_samples = tf.contrib.framework.nest.map_structure(
    lambda x: x.numpy(), iter(preprocessed_alice_dataset).next()
)

def model_instance():
    """Instantiates the keras model."""
    keras_model = create_compiled_keras_model()
    # training model by adding model and samples
    return tff.learning.from_compiled_keras_model(keras_model, batch_of_samples)

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [ ]:
from tensorflow_federated import python as tff

federated_learning_iterative_process = tff.learning.build_federated_averaging_process(
    model_instance
)
state = federated_learning_iterative_process.initialize()
state, performance = federated_learning_iterative_process.next(state, federated_data)

In [ ]:
performance